In [1]:
import os
from osgeo import gdal, osr, ogr
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from pyproj import Transformer
from rasterio.mask import mask
from shapely.geometry import box
from rasterio import features
import geopandas as gpd
import subprocess
from joblib import Parallel, delayed

output_file = '/home/karim/WaPOR/data/algeria/outputs/tmp1/reproj_'
input_file = '/home/karim/WaPOR/data/algeria/outputs/tmp1/reproj_cropped_'
resampled_file = "/home/karim/WaPOR/data/algeria/outputs/tmp1/resampled_output_"
my_shpfile = "/home/karim/WaPOR/data/algeria/segmentation/24tiles/V3/SNIC_30000_V2.shp"

start_year = 2018
end_year = 2024

# Define the list of dekads using list comprehension
deks = []

for year in range(start_year, end_year + 1):
    end_month = 12 if year <= 2023 else 3
    for month in range(1, end_month + 1):
        # For the first three months of each year, include all three dekads
        if month <= 3:
            for day in range(1, 4):
                deks.append(f"{year}-{month:02}-D{day}")
        # For the remaining months, include only the third dekad
        else:
            deks.append(f"{year}-{month:02}-D3")

print(deks)
num_cores=12

def update_tif_with_mask(tif_file_path, mask_file_path, output_file_path, mask_value=-1):
    # Open the GeoTIFF file
    with rasterio.open(tif_file_path, 'r') as tif_dataset:
        # Read the data as a NumPy array
        tif_data = tif_dataset.read(1)
        
        # Set all values below -1 to nodata
        nodata_value = tif_dataset.profile['nodata']
        tif_data[tif_data < -1] = nodata_value

        # Open the mask file
        with rasterio.open(mask_file_path, 'r') as mask_dataset:
            # Read the mask data as a NumPy array
            mask_data = mask_dataset.read(1)

            # Check if the extent, projection, and pixel numbers match
            if tif_dataset.bounds == mask_dataset.bounds and \
               tif_dataset.crs == mask_dataset.crs and \
               tif_dataset.width == mask_dataset.width and \
               tif_dataset.height == mask_dataset.height:

                # Replace pixels in the TIF file where the mask is equal to the specified value with NaN
                tif_data[mask_data == mask_value] = nodata_value

                # Create a new GeoTIFF file with updated values
                with rasterio.open(output_file_path, 'w', **tif_dataset.profile) as output_dataset:
                    output_dataset.write(tif_data, 1)

                print("Operation completed successfully.")
            else:
                print("Error: The extent, projection, or pixel numbers of the two datasets do not match.")

def avg_seg(filein, fileout):
    with rasterio.open(filein) as src:
        transform = src.transform
        dtype = src.dtypes[0]
        nodata = -9999  # Set nodata value to -9999
        height, width = src.height, src.width
        shapefile = gpd.read_file(my_shpfile)
        shapefile = shapefile.to_crs(src.crs)
        shapefile_bounds = shapefile.total_bounds
        shapefile_polygon = box(*shapefile_bounds)
        raster_cropped, transform_cropped = mask(src, [shapefile_polygon], crop=True)
        height_cropped, width_cropped = raster_cropped.shape[1:]
        raster_data_cropped = raster_cropped[0]
        raster_data_cropped[raster_data_cropped < -1] = nodata  # Set all values below -1 to nodata
        output_data = np.full((height_cropped, width_cropped), nodata, dtype=dtype)
        for idx, feature in shapefile.iterrows():
            geometry = feature.geometry
            feature_mask = features.geometry_mask([geometry], out_shape=(height_cropped, width_cropped), transform=transform_cropped)
            masked_values = np.ma.array(raster_data_cropped, mask=~feature_mask, fill_value=nodata)
            if np.any(masked_values.mask == True):
                average_value = np.nanmean(masked_values.data[masked_values.mask == True])
            else:
                average_value = nodata
            output_data[masked_values.mask == True] = average_value
        with rasterio.open(fileout, 'w', driver='GTiff', height=height_cropped,
                           width=width_cropped, count=1, dtype=dtype, nodata=nodata, crs=src.crs,
                           transform=transform_cropped) as dst:
            dst.write(output_data, 1)    
    raster_cropped = None
    shapefile = None

def process_nmfic(nmfic):
    input_raw_file = '/home/karim/WaPOR/data/algeria/AET_dekad/WAPOR-3.L3-AETI-D.MIT.'+nmfic[0:11]+'.tif'
    final_file = '/home/karim/WaPOR/data/algeria/outputs/AET_dekad/WAPOR-3.L3-AETI-D.MIT.'+nmfic[0:11]+'.tif'
    
    print('Traitement de ...', input_raw_file)
    src_srs = osr.SpatialReference()
    src_srs.ImportFromEPSG(4326)
    target_srs = osr.SpatialReference()
    target_srs.ImportFromEPSG(32631)
    try:
        input_ds = gdal.Open(input_raw_file)
        if input_ds is not None:
            output_file1=output_file+nmfic+'.tif'
            reproj_ds = gdal.Warp(output_file1, input_ds, format='GTiff', dstSRS=target_srs)
            geotransform = reproj_ds.GetGeoTransform()
            input_ds = None
            reproj_ds = None
        else:
            print("Failed to open the input file.")

    except Exception as e:
        print("An error occurred:", str(e))
    shapefile_ds = ogr.Open(my_shpfile)
    layer = shapefile_ds.GetLayer()
    extent = layer.GetExtent()
    output_raster = input_file+nmfic+'.tif' #"/home/wapor/reproj_cropped.tif"  # Save under /home/wapor directory
    output_file1=output_file+nmfic+'.tif'
    gdal.Warp(
        output_raster,
        output_file1,
        format="GTiff",
        outputBounds=(extent[0], extent[2], extent[1], extent[3]),  # Use the shapefile's bounding box
    )
    shapefile_ds = None
    #output_raster = None
    #output_raster = None
    snic_ds = gdal.Open(my_shpfile.replace(".shp", ".tif"))  # Assuming it's a GeoTIFF
    target_resolution = snic_ds.GetGeoTransform()[1]
    output_raster = gdal.Open(output_raster, gdal.GA_Update)
    #input_file1=input_file+nmfic+'.tif'
    #input_ds = gdal.Open(input_file1, gdal.GA_Update)
    resampled_file1 = resampled_file + nmfic + '.tif'
    try:
        gdal.Warp(
        resampled_file1,
        output_raster,
        format="GTiff",
        outputBounds=(extent[0], extent[2], extent[1], extent[3]),
        xRes=target_resolution,
        yRes=target_resolution,
        creationOptions=["TILED=YES", "COMPRESS=LZW"]
    )
    except Exception as e:
        print("An error occurred:", str(e))
    avg_seg(resampled_file1, final_file)
    output_raster = None
    os.remove(output_file1)
    #os.remove(output_raster)
    os.remove(resampled_file1)

if __name__ == '__main__':
    # Use joblib to parallelize the processing of deks1[0] and deks1[1] on separate cores.
    Parallel(n_jobs=num_cores)(delayed(process_nmfic)(nmfic) for nmfic in deks)

['2018-01-D1', '2018-01-D2', '2018-01-D3', '2018-02-D1', '2018-02-D2', '2018-02-D3', '2018-03-D1', '2018-03-D2', '2018-03-D3', '2018-04-D3', '2018-05-D3', '2018-06-D3', '2018-07-D3', '2018-08-D3', '2018-09-D3', '2018-10-D3', '2018-11-D3', '2018-12-D3', '2019-01-D1', '2019-01-D2', '2019-01-D3', '2019-02-D1', '2019-02-D2', '2019-02-D3', '2019-03-D1', '2019-03-D2', '2019-03-D3', '2019-04-D3', '2019-05-D3', '2019-06-D3', '2019-07-D3', '2019-08-D3', '2019-09-D3', '2019-10-D3', '2019-11-D3', '2019-12-D3', '2020-01-D1', '2020-01-D2', '2020-01-D3', '2020-02-D1', '2020-02-D2', '2020-02-D3', '2020-03-D1', '2020-03-D2', '2020-03-D3', '2020-04-D3', '2020-05-D3', '2020-06-D3', '2020-07-D3', '2020-08-D3', '2020-09-D3', '2020-10-D3', '2020-11-D3', '2020-12-D3', '2021-01-D1', '2021-01-D2', '2021-01-D3', '2021-02-D1', '2021-02-D2', '2021-02-D3', '2021-03-D1', '2021-03-D2', '2021-03-D3', '2021-04-D3', '2021-05-D3', '2021-06-D3', '2021-07-D3', '2021-08-D3', '2021-09-D3', '2021-10-D3', '2021-11-D3', '2021

/home/karim/miniconda3/lib/python3.12/site-packages/osgeo/osr.py:410: FutureWarning: Neither osr.UseExceptions() nor osr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
/home/karim/miniconda3/lib/python3.12/site-packages/osgeo/osr.py:410: FutureWarning: Neither osr.UseExceptions() nor osr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
/home/karim/miniconda3/lib/python3.12/site-packages/osgeo/osr.py:410: FutureWarning: Neither osr.UseExceptions() nor osr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
/home/karim/miniconda3/lib/python3.12/site-packages/osgeo/osr.py:410: FutureWarning: Neither osr.UseExceptions() nor osr.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
/home/karim/miniconda3/lib/python3.12/site-packa

Traitement de ... /home/karim/WaPOR/data/algeria/AET_dekad/WAPOR-3.L3-AETI-D.MIT.2018-02-D3.tif
Traitement de ... /home/karim/WaPOR/data/algeria/AET_dekad/WAPOR-3.L3-AETI-D.MIT.2019-01-D2.tif
Traitement de ... /home/karim/WaPOR/data/algeria/AET_dekad/WAPOR-3.L3-AETI-D.MIT.2020-01-D1.tif
Traitement de ... /home/karim/WaPOR/data/algeria/AET_dekad/WAPOR-3.L3-AETI-D.MIT.2020-12-D3.tif
Traitement de ... /home/karim/WaPOR/data/algeria/AET_dekad/WAPOR-3.L3-AETI-D.MIT.2021-09-D3.tif
Traitement de ... /home/karim/WaPOR/data/algeria/AET_dekad/WAPOR-3.L3-AETI-D.MIT.2022-07-D3.tif
Traitement de ... /home/karim/WaPOR/data/algeria/AET_dekad/WAPOR-3.L3-AETI-D.MIT.2023-06-D3.tif
Traitement de ... /home/karim/WaPOR/data/algeria/AET_dekad/WAPOR-3.L3-AETI-D.MIT.2018-02-D2.tif
Traitement de ... /home/karim/WaPOR/data/algeria/AET_dekad/WAPOR-3.L3-AETI-D.MIT.2019-01-D3.tif
Traitement de ... /home/karim/WaPOR/data/algeria/AET_dekad/WAPOR-3.L3-AETI-D.MIT.2020-01-D3.tif
Traitement de ... /home/karim/WaPOR/data